In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!


StatementMeta(, 695c5d9d-f56c-4b83-abab-5a4117f3c15c, 3, Finished, Available, Finished)

In [2]:
from pyspark.sql import functions as f
from pyspark.sql.window import Window 

StatementMeta(, 695c5d9d-f56c-4b83-abab-5a4117f3c15c, 4, Finished, Available, Finished)

In [3]:
df_customer= spark.table("silver_customers")

StatementMeta(, 695c5d9d-f56c-4b83-abab-5a4117f3c15c, 5, Finished, Available, Finished)

In [4]:
df_dim_exist=spark.catalog.tableExists("dim_customer")

StatementMeta(, 695c5d9d-f56c-4b83-abab-5a4117f3c15c, 6, Finished, Available, Finished)

In [5]:
window_desc=Window.partitionBy("customer_id").orderBy(f.col("updated_ts").desc())
window_asc=Window.partitionBy("customer_id").orderBy(f.col("updated_ts").asc())

StatementMeta(, 695c5d9d-f56c-4b83-abab-5a4117f3c15c, 7, Finished, Available, Finished)

In [6]:
df_ranked=(
     df_customer
    .withColumn("rn_desc",f.row_number().over(window_desc))
    .withColumn("rn_asc",f.row_number().over(window_asc))
)

StatementMeta(, 695c5d9d-f56c-4b83-abab-5a4117f3c15c, 8, Finished, Available, Finished)

In [7]:
display(df_ranked)

StatementMeta(, 695c5d9d-f56c-4b83-abab-5a4117f3c15c, 9, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 42f2246e-d945-44ed-81b8-b7dec78c0bd7)

In [8]:
df_dim_filter=(
    df_ranked
    .withColumn("is_current",
    f.when( f.col("rn_desc")==1 ,f.lit(1))
    .otherwise(0))
    )

StatementMeta(, 695c5d9d-f56c-4b83-abab-5a4117f3c15c, 10, Finished, Available, Finished)

In [9]:
df_scd_columns=(
    df_dim_filter
    .withColumn("customer_sk",f.monotonically_increasing_id())
    .withColumn("effective_startdate",f.to_date("updated_ts"))
    .withColumn(
        "effective_enddate",
    f.when(
    f.col("is_current")==1,
    f.lit("9999-12-31").cast("date")
    ).otherwise(
    f.lead(f.to_date("updated_ts")).over(window_asc)
    )
    )
    .drop("rn_desc","rn_asc")
    )

StatementMeta(, 695c5d9d-f56c-4b83-abab-5a4117f3c15c, 11, Finished, Available, Finished)

In [10]:
display(df_scd_columns)

StatementMeta(, 695c5d9d-f56c-4b83-abab-5a4117f3c15c, 12, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, d7924c14-a66e-43f4-b0e8-b234320286a1)

In [13]:
(
    df_scd_columns
    .write
    .mode("overwrite")
    .format("delta")
    .saveAsTable("dim_customer")
)

StatementMeta(, 695c5d9d-f56c-4b83-abab-5a4117f3c15c, 15, Finished, Available, Finished)